In [ ]:
print("hello world")

: 

In [ ]:
       # Use CSS selector to target the div with class "card"
#         box = response.css(".card")
# #         # Extract text from the div within the box with id "card-id"
#         box = box.css('.card-body')
#         box_text =box.css('div:nth-child(2)::text').getall()
#         print(" box dat is : ",box_text)
#         # Extract group_id
#         group_id = box_text[1].strip() if len(box_text) > 1 else None

#         # Safely extract created_at
#         created_at = box_text[11].strip() if len(box_text) > 11 else None

#         # Safely extract last_modified
#         last_modified = box_text[13].strip() if len(box_text) > 13 else None

#         yield {
#                 'group_id': group_id,
#                 'created_at': created_at,
#                 'last_modified': last_modified
#             }